In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 999)
import timeit

In [2]:
#fast way, took 81 sec for 500k rows (9 cols)
start_time = timeit.default_timer()
df= pd.read_csv('Fire_Dep_Call_20%.csv')
cols=['Call Date', 'Received DtTm', 'Entry DtTm', 'Dispatch DtTm', 'Response DtTm', 
              'On Scene DtTm', 'Transport DtTm', 'Hospital DtTm', 'Available DtTm']
def date_parser(cols, df):
    for col in cols:
        df[col] = pd.to_datetime(df[col],format='%m/%d/%Y %I:%M:%S %p',infer_datetime_format=True)
date_parser(cols, df)
elapsed = timeit.default_timer() - start_time
print('It took:'+ elapsed + 'to run it.')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
#very slow method, takes 660 sec for 500k rows (9 cols)
'''
start_time = timeit.default_timer()
df=pd.read_csv('Fire_Dep_Call_20%.csv', parse_dates=['Call Date', 'Received DtTm', 'Entry DtTm', 'Dispatch DtTm', 
            'Response DtTm', 'On Scene DtTm', 'Transport DtTm', 'Hospital DtTm', 'Available DtTm'],
            infer_datetime_format=True)
elapsed = timeit.default_timer() - start_time 
print('It took:'+ elapsed + 'to run it.')
'''

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
df.head(3)

,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,On Scene DtTm,Transport DtTm,Hospital DtTm,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,Original Priority,Priority,Final Priority,ALS Unit,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,Location,RowID,Call to Dispatch,Respond to On Scene,Response to Availiable,Respond to Availiable,On Scene to Availiable,weekday,month,year
0,1030101,E18,306091,Medical Incident,2000-04-12,04/12/2000,2000-04-12 21:00:29,2000-04-12 21:01:40,2000-04-12 21:02:00,NaT,NaT,NaT,NaT,Other,NaT,2000 Block of 37TH AVE,SF,94116.0,B08,18.0,757.0,3,3,3,False,NaN,1,ENGINE,1,8,4,Sunset/Parkside,"(37.7487247711275, -122.495504020186)",001030101-E18,00:01:31,NaT,NaT,NaT,NaT,2,2000,2000
1,1030104,M14,30612,Medical Incident,2000-04-12,04/12/2000,2000-04-12 21:09:02,2000-04-12 21:10:17,2000-04-12 21:10:29,2000-04-12 21:12:11,2000-04-12 21:19:36,2000-04-12 21:43:57,2000-04-12 22:03:33,Other,2000-04-12 22:23:57,1700 Block of 43RD AVE,SF,94122.0,B08,23.0,7651.0,3,3,3,True,NaN,1,MEDIC,2,8,4,Sunset/Parkside,"(37.7540326780595, -122.502185504543)",001030104-M14,00:01:27,00:07:25,01:11:46,01:11:46,01:04:21,2,2000,2000
2,1030106,M36,30614,Medical Incident,2000-04-12,04/12/2000,2000-04-12 21:09:44,2000-04-12 21:10:56,2000-04-12 21:11:47,NaT,2000-04-12 21:14:11,2000-04-12 21:59:31,2000-04-12 22:10:06,Other,2000-04-12 22:43:40,0 Block of FELL ST,SF,94102.0,B02,36.0,3111.0,3,3,3,False,NaN,1,MEDIC,1,2,6,Tenderloin,"(37.7764405100838, -122.418481123408)",001030106-M36,00:02:03,NaT,NaT,NaT,01:29:29,2,2000,2000


# Time NAs

In [14]:
#Creating time columns and boolean features  
#create col Call to Dispatch
df['Call to Dispatch'] = df['Dispatch DtTm']-df['Received DtTm']
#create col Respond to Availiable
df['Respond to Available'] = df['Available DtTm']-df['Response DtTm']
#create col Respond to On Scene
df['Respond to On Scene'] = df['On Scene DtTm']-df['Response DtTm']
#create col On Scene to Availiable
df['On Scene to Available'] = df['Available DtTm']-df['On Scene DtTm']
#create col Total Call to Availiable
df['Total Call to Available'] = df['Available DtTm']-df['Received DtTm']
#column to indicate if unit had transport or hospital part
df['Hospital or Transport'] = ((df['Transport DtTm'].isna()) | (df['Hospital DtTm'].isna()))*1

In [124]:
df.head()

,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,On Scene DtTm,Transport DtTm,Hospital DtTm,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,Original Priority,Priority,Final Priority,ALS Unit,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,Location,RowID,Call to Dispatch,Respond to On Scene,Response to Availiable,Respond to Availiable,On Scene to Availiable,Total Call to Available,Hospital or Transport
0,1030101,E18,306091,Medical Incident,2000-04-12,04/12/2000,2000-04-12 21:00:29,2000-04-12 21:01:40,2000-04-12 21:02:00,NaT,NaT,NaT,NaT,Other,NaT,2000 Block of 37TH AVE,SF,94116.0,B08,18.0,757.0,3,3,3,False,NaN,1,ENGINE,1,8,4,Sunset/Parkside,"(37.7487247711275, -122.495504020186)",001030101-E18,00:01:31,NaT,NaT,NaT,NaT,NaT,1
1,1030104,M14,30612,Medical Incident,2000-04-12,04/12/2000,2000-04-12 21:09:02,2000-04-12 21:10:17,2000-04-12 21:10:29,2000-04-12 21:12:11,2000-04-12 21:19:36,2000-04-12 21:43:57,2000-04-12 22:03:33,Other,2000-04-12 22:23:57,1700 Block of 43RD AVE,SF,94122.0,B08,23.0,7651.0,3,3,3,True,NaN,1,MEDIC,2,8,4,Sunset/Parkside,"(37.7540326780595, -122.502185504543)",001030104-M14,00:01:27,00:07:25,01:11:46,01:11:46,01:04:21,01:14:55,0
2,1030106,M36,30614,Medical Incident,2000-04-12,04/12/2000,2000-04-12 21:09:44,2000-04-12 21:10:56,2000-04-12 21:11:47,NaT,2000-04-12 21:14:11,2000-04-12 21:59:31,2000-04-12 22:10:06,Other,2000-04-12 22:43:40,0 Block of FELL ST,SF,94102.0,B02,36.0,3111.0,3,3,3,False,NaN,1,MEDIC,1,2,6,Tenderloin,"(37.7764405100838, -122.418481123408)",001030106-M36,00:02:03,NaT,NaT,NaT,01:29:29,01:33:56,0
3,1030107,E01,30615,Alarms,2000-04-12,04/12/2000,2000-04-12 21:13:47,2000-04-12 21:13:51,2000-04-12 21:14:13,2000-04-12 21:15:58,2000-04-12 21:20:12,NaT,NaT,Other,2000-04-12 21:24:19,100 Block of JONES ST,SF,94102.0,B03,1.0,1456.0,3,3,3,False,NaN,1,ENGINE,3,3,6,Tenderloin,"(37.7825474000421, -122.412247935495)",001030107-E01,00:00:26,00:04:14,00:08:21,00:08:21,00:04:07,00:10:32,1
4,1030108,RS1,30616,Medical Incident,2000-04-12,04/12/2000,2000-04-12 21:14:43,2000-04-12 21:16:11,2000-04-12 21:16:24,2000-04-12 21:18:20,2000-04-12 21:20:08,NaT,NaT,Other,2000-04-12 21:20:36,700 Block of MARKET ST,SF,94108.0,B03,1.0,1322.0,3,3,3,False,NaN,1,RESCUE SQUAD,2,1,3,Financial District/South Beach,"(37.7863072236365, -122.405294845215)",001030108-RS1,00:01:41,00:01:48,00:02:16,00:02:16,00:00:28,00:05:53,1


In [137]:
time_df = df[['Call Date', 'Received DtTm', 'Entry DtTm', 'Dispatch DtTm', 'Response DtTm', 
              'On Scene DtTm', 'Transport DtTm', 'Hospital DtTm', 'Available DtTm']]

In [141]:
time_df.isnull().sum()

Call Date              0
Received DtTm          0
Entry DtTm             0
Dispatch DtTm          0
Response DtTm      57737
On Scene DtTm     132521
Transport DtTm    362478
Hospital DtTm     415092
Available DtTm     12416
dtype: int64